## Данные и матрица интеракций

In [2]:
%pip install pandas

  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 66.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 21.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('recipes_normalized.csv')

data.head(10)

,url,name,ingredients,ingredients_normalized
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...","{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...","{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...","{'Фарш куриный': '800 г', 'Картофель': '1 кг',..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар-песок..."
5,https://www.povarenok.ru/recipes/show/167765/,Песочный пирог с тыквенным суфле,"{'Масло сливочное': '100 г', 'Сахар': '50 г', ...","{'Масло сливочное': '100 г', 'Сахар-песок': '5..."
6,https://www.povarenok.ru/recipes/show/100230/,Шоколадные конфеты ручной работы,"{'Какао-масло': '100 г', 'Какао тертое': '200 ...","{'Какао-масло': '100 г', 'Какао': '200 г', 'Си..."
7,https://www.povarenok.ru/recipes/show/96257/,Рыбно-тыквенный гратен,"{'Масло растительное': '3 ст. л.', 'Рыба': '40...","{'Масло растительное': '3 ст. л.', 'Рыба': '40..."
8,https://www.povarenok.ru/recipes/show/139360/,"Плов с креветками из риса ""Басмати""","{'Рис': '2 стак.', 'Вода': '4 стак.', 'Морковь...","{'Рис': '2 стак.', 'Вода': '4 стак.', 'Морковь..."
9,https://www.povarenok.ru/recipes/show/96774/,Молочное мороженое,"{'Молоко': '450 г', 'Желток яичный': '3 шт', '...","{'Молоко': '450 г', 'Желтки': '3 шт', 'Сахар-п..."


### Почистим данные 

In [6]:
%pip install scipy scikit-learn

  Using cached scipy-1.16.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 55.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 56.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 14.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder
import ast

all_ingredients = set()
for ingredients_str in data['ingredients_normalized']:
    ingredients_parsed = ast.literal_eval(ingredients_str)
    all_ingredients.update(ingredients_parsed.keys())

all_ingredients = sorted(list(all_ingredients))
ingredient_to_idx = {ing: idx for idx, ing in enumerate(all_ingredients)}

print(f"Ингредиентов: {len(all_ingredients)}")
print(f"Рецептов: {len(data)}")

Ингредиентов: 979
Рецептов: 146581


In [8]:
stop_words_drop = ['Соль', 'Сахар-песок', 'Перец черный молотый', 'Мука пшеничная',
'Сода', 'Сода гашеная уксусом']

In [9]:
from tqdm import tqdm

interactions = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    ingredients_parsed = ast.literal_eval(row['ingredients_normalized'])
    recipe_id = row.get('url', idx) 
        
    for ingredient in ingredients_parsed.keys():
        interactions.append((recipe_id, ingredient))

interactions_df = pd.DataFrame(interactions, columns=['recipe_id', 'ingredient_id'])
print(f"Interactions {len(interactions_df)}")

unique_recipes = interactions_df['recipe_id'].unique()
all_unique_ingredients = interactions_df['ingredient_id'].unique()
unique_ingredients = [
    ingredient for ingredient in all_unique_ingredients if ingredient not in stop_words_drop
]

recipe2id = {recipe: i for i, recipe in enumerate(unique_recipes)}
item2id = {ingredient: i for i, ingredient in enumerate(unique_ingredients)}

id2recipe = {i: recipe for recipe, i in recipe2id.items()}
id2item = {i: ingredient for ingredient, i in item2id.items()}

interactions_df['user_id'] = interactions_df['recipe_id'].map(recipe2id)
interactions_df['item_id'] = interactions_df['ingredient_id'].map(item2id)
interactions_df.dropna(subset=['item_id'], inplace=True)
interactions_df['item_id'] = interactions_df['item_id'].astype(int)


  2%|▏         | 2643/146581 [00:00<00:10, 13267.82it/s]

100%|██████████| 146581/146581 [00:11<00:00, 12755.72it/s]


Interactions 1278324


In [10]:
print(len(unique_recipes), len(unique_ingredients))

146563 973


In [11]:
interactions_df.head(7)

,recipe_id,ingredient_id,user_id,item_id
0,https://www.povarenok.ru/recipes/show/164365/,Молоко,0,0
1,https://www.povarenok.ru/recipes/show/164365/,Клубника,0,1
3,https://www.povarenok.ru/recipes/show/1306/,Сыр твердый,1,2
4,https://www.povarenok.ru/recipes/show/1306/,Чеснок,1,3
5,https://www.povarenok.ru/recipes/show/1306/,Яйцо куриное,1,4
6,https://www.povarenok.ru/recipes/show/1306/,Грейпфрут,1,5
7,https://www.povarenok.ru/recipes/show/1306/,Лук зеленый,1,6


In [12]:
rows = interactions_df['user_id'].values
cols = interactions_df['item_id'].values
values = np.ones(len(rows), dtype=np.int8)

num_users= len(unique_recipes)
num_items= len(unique_ingredients)

interactions_matrix = csr_matrix(
    (values, (rows, cols)), 
    shape=(num_users, num_items)
)

In [13]:
print(f"Sparsity:{(1 -interactions_matrix.nnz /(num_users * num_items)) * 100:.2f}%")

Sparsity:99.26%


In [14]:
interactions_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(146563, 973), dtype=int8)

### Train test split

In [15]:
def train_val_test_split(
    interactions_df: pd.DataFrame,
    user_col: str = 'user_id',
    item_col: str = 'item_id',
    k_core: int = 3,
    test_size: float = 0.2,
    val_size: float = 0.1,
    holdout_frac_range: tuple[float, float] = (0.3, 0.4),
    seed: int = 42
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    filtered_df = interactions_df.copy()

    while True:
        user_counts = filtered_df.groupby(user_col)[item_col].count()
        valid_users = user_counts[user_counts >= k_core].index

        item_counts = filtered_df.groupby(item_col)[user_col].count()
        valid_items = item_counts[item_counts >= k_core].index

        before_count = len(filtered_df)
        filtered_df = filtered_df[
            (filtered_df[user_col].isin(valid_users)) &
            (filtered_df[item_col].isin(valid_items))
        ]
        after_count = len(filtered_df)

        if before_count == after_count:
            break

    print(f"interactions left after k-core filtering: {len(filtered_df)}")

    rng = np.random.default_rng(seed)
    shuffled_users = rng.permutation(filtered_df[user_col].unique())

    n_users = len(shuffled_users)
    n_test = int(n_users * test_size)
    n_val = int(n_users * val_size)

    test_users = shuffled_users[:n_test]
    val_users = shuffled_users[n_test:n_test + n_val]
    train_users = shuffled_users[n_test + n_val:]

    train_df = filtered_df[filtered_df[user_col].isin(train_users)].copy()
    val_df = filtered_df[filtered_df[user_col].isin(val_users)].copy()
    test_df = filtered_df[filtered_df[user_col].isin(test_users)].copy()

    def _support_holdout(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
        if df.empty:
            return df.copy(), df.copy()
        support_rows = []
        holdout_rows = []
        for _, user_df in df.groupby(user_col):
            user_df = user_df.sample(frac=1, random_state=rng.integers(0, 1_000_000))
            holdout_share = rng.uniform(*holdout_frac_range)
            n_holdout = min(len(user_df) - 1, max(1, int(round(len(user_df) * holdout_share))))
            holdout_rows.append(user_df.iloc[:n_holdout])
            support_rows.append(user_df.iloc[n_holdout:])
        return (
            pd.concat(support_rows).reset_index(drop=True),
            pd.concat(holdout_rows).reset_index(drop=True)
        )

    val_support_df, val_holdout_df = _support_holdout(val_df)
    test_support_df, test_holdout_df = _support_holdout(test_df)

    return (
        train_df.reset_index(drop=True),
        val_support_df,
        val_holdout_df,
        test_support_df,
        test_holdout_df,
    )

In [16]:
train_df, val_support_df, val_holdout_df, test_support_df, test_holdout_df = train_val_test_split(
    interactions_df, 
    k_core=5, 
    test_size=0.2,
    val_size=0.15   
)

interactions left after k-core filtering: 978048


#### for eval

In [17]:
split_frames = {
    'train': (train_df, None),
    'val': (val_support_df, val_holdout_df),
    'test': (test_support_df, test_holdout_df),
}

rows = []
for split_name, (support_df, holdout_df) in split_frames.items():
    support_grp = support_df.groupby('user_id')['item_id'].apply(list)
    holdout_grp = (
        holdout_df.groupby('user_id')['item_id'].apply(list)
        if holdout_df is not None and not holdout_df.empty else pd.Series(dtype=object)
    )
    combined_users = support_grp.index.union(holdout_grp.index)
    for user_id in combined_users:
        rows.append({
            'user_id': user_id,
            'split': split_name,
            'support_items': support_grp.get(user_id, []),
            'holdout_items': holdout_grp.get(user_id, []),
        })

full_grouped_data = pd.DataFrame(rows)

In [44]:
import random
import math

def convert_ids_to_strings(grouped_df, id2item, id2recipe):
    """
    Convert IDs to readable strings in the grouped dataframe
    """
    df_copy = grouped_df.copy()
    
    # Convert user_id (recipe IDs) to recipe URLs/names if needed
    # df_copy['recipe_name'] = df_copy['user_id'].map(id2recipe)
    
    # Convert support_items (ingredient IDs) to ingredient names
    df_copy['support_items_names'] = df_copy['support_items'].apply(
        lambda item_ids: [id2item.get(i, f'Unknown_{i}') for i in item_ids]
    )
    
    # Convert holdout_items (ingredient IDs) to ingredient names  
    df_copy['holdout_items_names'] = df_copy['holdout_items'].apply(
        lambda item_ids: [id2item.get(i, f'Unknown_{i}') for i in item_ids]
    )
    
    # For train split, move 30% of ingredients from support to holdout
    random.seed(42)  # Фиксируем сид для воспроизводимости
    train_mask = df_copy['split'] == 'train'
    
    for idx in df_copy[train_mask].index:
        support_items = df_copy.loc[idx, 'support_items_names']
        if len(support_items) > 0:  # Добавляем проверку на пустой список
            n_move = max(1, math.ceil(0.3 * len(support_items)))  # Минимум 1 элемент
            
            if n_move < len(support_items):
                moved_items = random.sample(support_items, n_move)
                # Используем .at вместо .loc для установки значений
                df_copy.at[idx, 'support_items_names'] = [item for item in support_items if item not in moved_items]
                df_copy.at[idx, 'holdout_items_names'] = moved_items
    
    return df_copy

# Apply the conversion
full_grouped_data_with_names = convert_ids_to_strings(full_grouped_data, id2item, id2recipe)
full_grouped_data_with_names = full_grouped_data_with_names.drop(["support_items", 'holdout_items'], axis=1)
full_grouped_data_with_names["support_items"] = full_grouped_data_with_names["support_items_names"]
full_grouped_data_with_names["holdout_items"] = full_grouped_data_with_names["holdout_items_names"]
full_grouped_data_with_names = full_grouped_data_with_names.drop(["support_items_names", 'holdout_items_names'], axis=1)
# Now you can inspect the data with readable names
#print(full_grouped_data_with_names[['user_id', 'split', 'support_items_names', 'holdout_items_names']].head())

In [45]:
full_grouped_data_bebra

,user_id,split,support_items,holdout_items
0,1,train,"[Сыр твердый, Чеснок, Яйцо куриное, Грейпфрут,...",[]
1,2,train,"[Баклажан, Лук репчатый, Помидор, Чеснок, Майо...",[]
2,4,train,"[Вишня, Водка, Кости, Корица, Гвоздика]",[]
3,5,train,"[Масло сливочное, Яйцо куриное, Вода, Тыква, М...",[]
4,7,train,"[Масло растительное, Рыба, Тыква, Лук репчатый...",[]
...,...,...,...,...
123303,146522,test,"[Огурец, Креветки, Чеснок, Перец болгарский, С...","[Кунжут, Соевый соус, Масло растительное]"
123304,146523,test,"[Зелень, Сметана, Сыр колбасный, Масло растите...","[Кабачок, Багет, Лук зеленый, Творог]"
123305,146527,test,"[Крахмал картофельный, Молоко, Яйцо куриное, В...","[Масло сливочное, Сахар тростниковый]"
123306,146535,test,"[Масло растительное, Треска, Молоко, Сок лимона]","[Мед, Соевый соус]"


In [47]:
full_grouped_data_with_names.to_csv("data-ready_lmm_train.csv")

In [29]:
full_grouped_data_with_names.to_csv("data_llm.csv")

In [ ]:
pd.to_csv()

In [58]:
from math import log2

def hit_rate(recommendations, ground_truth, k=100):
    """
    Calculate HitRate@k

    Args:
        recommendations: dict {user_id: list of recommended item_ids}
        ground_truth: dict {user_id: set of relevant item_ids}
        k: cutoff level
    """
    hits = 0
    total_users = len(recommendations)

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        if any(item in user_truth for item in user_recs):
            hits += 1

    return hits / total_users if total_users > 0 else 0.0


def precision(recommendations, ground_truth, k=100):
    """
    Calculate Precision@k
    """
    precisions = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())
        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_precision = relevant_count / k
        precisions.append(user_precision)

    return np.mean(precisions) if precisions else 0.0


def recall(recommendations, ground_truth, k=100):
    """
    Calculate Recall@k
    """
    recalls = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        if not user_truth:  # If no ground truth items, recall is 0
            recalls.append(0.0)
            continue

        relevant_count = sum(1 for item in user_recs if item in user_truth)
        user_recall = relevant_count / len(user_truth)
        recalls.append(user_recall)

    return np.mean(recalls) if recalls else 0.0

def mrr(recommendations, ground_truth, k=100):
    """
    Calculate MRR@k
    """
    reciprocal_ranks = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, set())

        user_rr = 0.0
        for rank, item in enumerate(user_recs, 1):
            if item in user_truth:
                user_rr = 1.0 / rank
                break

        reciprocal_ranks.append(user_rr)

    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0

def ndcg(recommendations, ground_truth, k=100, binary_relevance=True):
    """
    Calculate NDCG@k

    Args:
        binary_relevance: if True, uses binary relevance (0/1),
                         if False, expects relevance scores in ground_truth
    """
    ndcg_scores = []

    for user_id, recs in recommendations.items():
        user_recs = recs[:k]
        user_truth = ground_truth.get(user_id, {})

        # Calculate DCG
        dcg = 0.0
        for rank, item in enumerate(user_recs, 1):
            if binary_relevance:
                rel = 1.0 if item in user_truth else 0.0
            else:
                rel = user_truth.get(item, 0.0)

            dcg += rel / (log2(rank + 1))

        # Calculate IDCG
        if binary_relevance:
            # For binary relevance, ideal is all 1's sorted first
            num_relevant = len(user_truth)
            ideal_gains = [1.0] * min(k, num_relevant)
        else:
            # For graded relevance, take top-k relevance scores
            ideal_gains = sorted(user_truth.values(), reverse=True)[:k]

        idcg = 0.0
        for rank, rel in enumerate(ideal_gains, 1):
            idcg += rel / (log2(rank + 1))

        user_ndcg = dcg / idcg if idcg > 0 else 0.0
        ndcg_scores.append(user_ndcg)

    return np.mean(ndcg_scores) if ndcg_scores else 0.0
        

In [12]:
def evaluate_model(df: pd.DataFrame, preds_col: str, gt_col: str, top_k: int = 20) -> dict:
    eval_df = df[df[gt_col].map(len) > 0].copy()
    recommendations = dict(zip(eval_df['user_id'], eval_df[preds_col]))
    ground_truth = dict(zip(eval_df['user_id'], eval_df[gt_col].apply(set)))

    if not recommendations:
        return {metric: 0.0 for metric in (f'hit_rate@{top_k}', f'precision@{top_k}',
                                           f'recall@{top_k}', f'mrr@{top_k}', f'ndcg@{top_k}')}

    hr = hit_rate(recommendations, ground_truth, k=top_k)
    p = precision(recommendations, ground_truth, k=top_k)
    r = recall(recommendations, ground_truth, k=top_k)
    m = mrr(recommendations, ground_truth, k=top_k)
    n = ndcg(recommendations, ground_truth, k=top_k)

    return {
        f'hit_rate@{top_k}': hr,
        f'precision@{top_k}': p,
        f'recall@{top_k}': r,
        f'mrr@{top_k}': m,
        f'ndcg@{top_k}': n
    }

### model 1: EASE

In [14]:
from scipy import sparse as sps

matrix_train = sps.coo_matrix(
    (np.ones(train_df.shape[0]), (train_df['user_id'], train_df['item_id'])),
    shape=(len(recipe2id), len(item2id)),
)
matrix_train

<COOrdinate sparse matrix of dtype 'float64'
	with 636258 stored elements and shape (146563, 973)>

In [15]:
%%time

# Обучаем конечную модель
# Мы взяли реализацию из RecBole

def fit_ease(X, reg_weight=100):
    
    # gram matrix
    G = X.T @ X

    # add reg to diagonal
    G += reg_weight * sps.identity(G.shape[0])

    # convert to dense because inverse will be dense
    G = G.todense()

    # invert. this takes most of the time
    P = np.linalg.inv(G)
    B = P / (-np.diag(P))
    # zero out diag
    np.fill_diagonal(B, 0.)
    
    return B

w = fit_ease(matrix_train)

CPU times: user 197 ms, sys: 81.7 ms, total: 279 ms
Wall time: 973 ms


In [16]:
def get_preds(user_interactions, item2id, id2item, model_weights):
    encoded_ids = user_interactions
    
    vector = np.zeros(len(item2id))
    vector[encoded_ids] = 1
    
    preds = vector @ model_weights
    preds[encoded_ids] = -np.inf  # Filter out items already seen
    
    top_indices = np.argsort(-preds)[:20]
    
    decoded = [id2item[i] for i in top_indices]
    
    return  top_indices

In [17]:
w = np.asarray(w)

tqdm.pandas()
test_mask = full_grouped_data['split'] == 'test'
full_grouped_data.loc[test_mask, 'ease_preds'] = (
    full_grouped_data.loc[test_mask, 'support_items']
    .progress_apply(lambda interactions: get_preds(interactions, item2id, id2item, w))
)
full_grouped_data.loc[test_mask].tail()

100%|██████████| 24661/24661 [00:08<00:00, 2752.54it/s]


,user_id,split,support_items,holdout_items,ease_preds
123303,146522,test,"[55, 54, 3, 46, 68]","[210, 49, 18]","[18, 11, 8, 16, 49, 145, 71, 7, 122, 109, 77, ..."
123304,146523,test,"[16, 58, 290, 18, 15, 48, 11, 4, 3]","[213, 379, 6, 142]","[10, 0, 2, 65, 46, 13, 8, 24, 142, 9, 213, 151..."
123305,146527,test,"[288, 0, 4, 66, 24]","[15, 341]","[15, 18, 65, 151, 142, 29, 31, 202, 169, 14, 5..."
123306,146535,test,"[18, 329, 0, 68]","[159, 49]","[4, 10, 151, 15, 3, 49, 122, 48, 232, 64, 145,..."
123307,146547,test,"[278, 122, 36, 4]","[210, 49, 13]","[8, 15, 18, 6, 0, 10, 58, 145, 13, 55, 11, 65,..."


In [59]:
evaluate_model(full_grouped_data[full_grouped_data['split'] == 'test'], 'ease_preds', 'holdout_items', top_k=10)

{'hit_rate@10': 0.8350837354527392,
 'precision@10': np.float64(0.13921982076963627),
 'recall@10': np.float64(0.5034939171997685),
 'mrr@10': np.float64(0.5385431852748669),
 'ndcg@10': np.float64(0.4161305502593234)}

In [19]:
evaluate_model(full_grouped_data[full_grouped_data['split'] == 'test'], 'ease_preds', 'holdout_items', top_k=6)

{'hit_rate@6': 0.7558493167349256,
 'precision@6': np.float64(0.1867320871010908),
 'recall@6': np.float64(0.4069210558667596),
 'mrr@6': np.float64(0.5276942000189233),
 'ndcg@6': np.float64(0.40705026830488084)}

In [21]:
evaluate_model(full_grouped_data[full_grouped_data['split'] == 'test'], 'ease_preds', 'holdout_items', top_k=20)

{'hit_rate@20': 0.9161428976927132,
 'precision@20': np.float64(0.08818579944041198),
 'recall@20': np.float64(0.6349355192923986),
 'mrr@20': np.float64(0.5431274639393558),
 'ndcg@20': np.float64(0.6349355192923986)}

In [26]:
#навайбкодила как могла 
def inspect_recommendations(
    user_id: int,
    df: pd.DataFrame,
    id2recipe: dict,
    id2item: dict,
    preds: str,
    split: str = 'test',
    top_k: int | None = 20,
):
    user_mask = df['user_id'] == user_id
    if split is not None:
        user_mask &= df['split'] == split

    user_rows = df[user_mask]
    if user_rows.empty:
        print(f"No data found for user {user_id} in split '{split}'.")
        return

    user_data = user_rows.iloc[0]
    recipe_name = id2recipe.get(user_id, "Unknown Recipe")

    def _ensure_list(values):
        if values is None or (isinstance(values, float) and np.isnan(values)):
            return []
        return list(values)

    support_items = _ensure_list(user_data.get('support_items'))
    holdout_items = _ensure_list(user_data.get('holdout_items'))
    pred_items = _ensure_list(user_data.get(preds))

    if top_k is not None:
        pred_items = pred_items[:top_k]

    support_names = [id2item.get(i, "Unknown") for i in support_items]
    holdout_names = [id2item.get(i, "Unknown") for i in holdout_items]
    pred_names = [id2item.get(i, "Unknown") for i in pred_items]

    print("=" * 80)
    print(f"RECIPE: {recipe_name} (User ID: {user_id}) | split: {split}")
    print("=" * 80)

    print(f"Observed Ingredients ({len(support_names)})")
    print(", ".join(support_names))

    print(f"\nGround Truth Ingredients ({len(holdout_names)})")
    print(", ".join(holdout_names))

    print(f"\nTop Recommended Ingredients ({len(pred_names)})")
    successful_preds = set(holdout_names).intersection(set(pred_names))

    display_preds = []
    for item in pred_names:
        display_preds.append(item)

    print(", ".join(display_preds))
    print("\n" + "=" * 80)

example_test_user = full_grouped_data.loc[full_grouped_data['split'] == 'test', 'user_id'].iloc[0]
inspect_recommendations(
    user_id=example_test_user,
    df=full_grouped_data,
    id2recipe=id2recipe,
    id2item=id2item,
    preds='ease_preds',
    split='test'
)

RECIPE: https://www.povarenok.ru/recipes/show/34234/ (User ID: 16) | split: test
Observed Ingredients (5)
Лук репчатый, Майонез, Огурец, Фасоль, Картофель

Ground Truth Ingredients (3)
Морковь, Свекла, Горчица

Top Recommended Ingredients (20)
Морковь, Яйцо куриное, Зелень, Помидор, Чеснок, Масло растительное, Грибы, Сыр твердый, Шампиньоны, Укроп, Кукуруза, Горошек зеленый, Перец болгарский, Филе куриное, Специи, Колбаса, Лук зеленый, Томатная паста, Фарш, Свекла



In [55]:
random_test_user_id = full_grouped_data.loc[full_grouped_data['split'] == 'test', 'user_id'].sample(1).iloc[0]
inspect_recommendations(
    user_id=random_test_user_id,
    df=full_grouped_data,
    id2recipe=id2recipe,
    id2item=id2item,
    preds='ease_preds',
    split='test'
)

RECIPE: https://www.povarenok.ru/recipes/show/122812/ (User ID: 34866) | split: test
Observed Ingredients (8)
Имбирь, Блин, Крупа гречневая, Яйцо куриное, Морковь, Лук репчатый, Чеснок, Масло растительное

Ground Truth Ingredients (4)
Сметана, Кабачок, Молоко, Фарш

Top Recommended Ingredients (20)
Вода, Соевый соус, Сметана, Грибы, Зелень, Фарш, Масло сливочное, Перец чили, Молоко, Майонез, Шампиньоны, Перец болгарский, Творог, Корица, Специи, Сыр твердый, Мед, Филе куриное, Курица, Картофель



### hyperparameters tuning

In [27]:
import optuna

val_mask = full_grouped_data['split'] == 'val'
tqdm.pandas()

def objective(trial):
    regul = trial.suggest_int('regul', 50, 2000)
    w_candidate = np.asarray(fit_ease(matrix_train, regul))
    val_subset = full_grouped_data.loc[val_mask, ['user_id', 'support_items', 'holdout_items']].copy()
    val_subset['ease_val_preds'] = val_subset['support_items'].progress_apply(
        lambda interactions: get_preds(interactions, item2id, id2item, w_candidate)
    )
    return evaluate_model(val_subset, 'ease_val_preds', 'holdout_items', top_k=6)['hit_rate@6']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

/Users/polina/Desktop/sirius/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-11-15 11:21:21,095] A new study created in memory with name: no-name-e4dc7455-2114-4d33-9e1e-867749d9482e
100%|██████████| 18496/18496 [00:08<00:00, 2138.22it/s]
[I 2025-11-15 11:21:31,363] Trial 0 finished with value: 0.7490808823529411 and parameters: {'regul': 851}. Best is trial 0 with value: 0.7490808823529411.
100%|██████████| 18496/18496 [00:07<00:00, 2409.78it/s]
[I 2025-11-15 11:21:40,108] Trial 1 finished with value: 0.74405276816609 and parameters: {'regul': 1616}. Best is trial 0 with value: 0.7490808823529411.
100%|██████████| 18496/18496 [00:10<00:00, 1829.85it/s]
[I 2025-11-15 11:21:50,913] Trial 2 finished with value: 0.7428633217993079 and parameters: {'regul': 1863}. Best is trial 0 with value: 0

In [28]:
study.best_trial.params['regul']

77

results on train + val (optimal parameters)

In [29]:
train_val_df = pd.concat([train_df, val_support_df], ignore_index=True)

num_users = len(recipe2id)
num_items = len(item2id)

interactions_matrix_train_val = sps.coo_matrix(
    (np.ones(train_val_df.shape[0]), (train_val_df['user_id'], train_val_df['item_id'])),
    shape=(num_users, num_items),
).tocsr()

In [30]:
w_final = fit_ease(interactions_matrix_train_val, reg_weight=study.best_trial.params['regul'])
w_final_array = np.asarray(w_final)

tqdm.pandas()
for split_name in ['val', 'test']:
    split_mask = full_grouped_data['split'] == split_name
    if split_mask.any():
        full_grouped_data.loc[split_mask, 'ease_preds'] = (
            full_grouped_data.loc[split_mask, 'support_items']
            .progress_apply(lambda interactions: get_preds(interactions, item2id, id2item, w_final_array))
        )

100%|██████████| 24661/24661 [00:19<00:00, 1269.77it/s]


In [31]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='ease_preds',
    gt_col='holdout_items',
    top_k=6
)

{'hit_rate@6': 0.7556465674546855,
 'precision@6': np.float64(0.18732681832312825),
 'recall@6': np.float64(0.4078772600912822),
 'mrr@6': np.float64(0.5288235135098603),
 'ndcg@6': np.float64(0.40800454158387733)}

In [32]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'val'],
    preds_col='ease_preds',
    gt_col='holdout_items',
    top_k=6
)

{'hit_rate@6': 0.75967776816609,
 'precision@6': np.float64(0.18888768742791234),
 'recall@6': np.float64(0.41219652383012023),
 'mrr@6': np.float64(0.5327854671280277),
 'ndcg@6': np.float64(0.4123062644175317)}

In [33]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='ease_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.8350837354527392,
 'precision@10': np.float64(0.13921982076963627),
 'recall@10': np.float64(0.5034939171997685),
 'mrr@10': np.float64(0.5385431852748669),
 'ndcg@10': np.float64(0.5034939171997685)}

In [34]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'val'],
    preds_col='ease_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.8356942041522492,
 'precision@10': np.float64(0.1397923875432526),
 'recall@10': np.float64(0.5059610700898007),
 'mrr@10': np.float64(0.5421874570906794),
 'ndcg@10': np.float64(0.5059610700898007)}

## slim

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [36]:
from sklearn.linear_model import ElasticNet

def train_slim(
    train_matrix: sps.csr_matrix, 
    l1_reg: float = 0.001, 
    l2_reg: float = 0.0001
) -> sps.csr_matrix:
    num_items = train_matrix.shape[1]
    
    train_matrix_csc = train_matrix.tocsc()
    rows, cols, data = [], [], []

    model = ElasticNet(
        alpha=l1_reg + l2_reg,
        l1_ratio=l1_reg / (l1_reg + l2_reg) if (l1_reg + l2_reg) > 0 else 0,
        positive=True,
        fit_intercept=False,
        copy_X=False,   
        precompute=True, 
        max_iter=300, 
        tol=1e-4 
    )

    for j in tqdm(range(num_items)):
        y = train_matrix_csc[:, j].toarray().ravel()
        
        # (w_jj = 0) j-th column of the training data to zero
        start_pos = train_matrix_csc.indptr[j]
        end_pos = train_matrix_csc.indptr[j + 1]
        
        original_values = train_matrix_csc.data[start_pos:end_pos].copy()
        train_matrix_csc.data[start_pos:end_pos] = 0.0

        model.fit(train_matrix_csc, y)
        coeffs = model.coef_
        non_zero_indices = coeffs.nonzero()[0]
        if len(non_zero_indices) > 0:
            rows.extend(non_zero_indices)
            cols.extend([j] * len(non_zero_indices))
            data.extend(coeffs[non_zero_indices])
            
        train_matrix_csc.data[start_pos:end_pos] = original_values


    W_slim = sps.csr_matrix((data, (rows, cols)), shape=(num_items, num_items))
    
    return W_slim

In [37]:
w_slim = train_slim(matrix_train, l1_reg=0.001, l2_reg=0.001)

100%|██████████| 973/973 [04:01<00:00,  4.03it/s] 


In [39]:
def get_slim_predictions(user_interactions, model_weights, top_k=20):
    vector = np.zeros(model_weights.shape[1])
    vector[user_interactions] = 1
    
    scores = (sps.csr_matrix(vector) @ model_weights).toarray().flatten()
    
    scores[user_interactions] = -np.inf
    
    top_indices = np.argsort(-scores)[:top_k].tolist()
    
    return top_indices

In [40]:
tqdm.pandas()
for split_name in ['val', 'test']:
    split_mask = full_grouped_data['split'] == split_name
    if split_mask.any():
        full_grouped_data.loc[split_mask, 'slim_preds'] = (
            full_grouped_data.loc[split_mask, 'support_items']
            .progress_apply(lambda interactions: get_slim_predictions(interactions, w_slim))
        )

full_grouped_data.loc[full_grouped_data['split'] == 'test'].head()

  0%|          | 0/18496 [00:00<?, ?it/s]

100%|██████████| 24661/24661 [00:02<00:00, 8449.46it/s]


,user_id,split,support_items,holdout_items,ease_preds,slim_preds
98647,16,test,"[10, 8, 55, 63, 13]","[39, 62, 64]","[39, 4, 16, 3, 11, 18, 43, 2, 145, 70, 124, 48...","[39, 4, 18, 2, 16, 3, 11, 145, 48, 6, 70, 43, ..."
98648,21,test,"[15, 86, 85]","[24, 18]","[0, 24, 79, 65, 93, 4, 212, 142, 151, 159, 450...","[0, 4, 65, 35, 29, 24, 58, 14, 61, 66, 26, 31,..."
98649,26,test,"[61, 15, 102, 4]","[89, 68]","[0, 65, 29, 142, 58, 151, 31, 24, 35, 22, 93, ...","[65, 0, 58, 142, 29, 31, 35, 24, 8, 66, 18, 2,..."
98650,30,test,"[4, 18, 70, 39, 10]","[15, 107]","[13, 8, 0, 65, 2, 58, 24, 16, 15, 38, 3, 164, ...","[13, 3, 15, 24, 8, 0, 2, 58, 48, 16, 65, 46, 3..."
98651,31,test,"[48, 18, 16, 108, 109]","[54, 111, 110]","[68, 11, 55, 10, 3, 46, 111, 49, 54, 39, 77, 2...","[10, 3, 68, 13, 4, 2, 24, 11, 39, 46, 49, 77, ..."


In [61]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='slim_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.790722192936215,
 'precision@10': np.float64(0.1242569238879202),
 'recall@10': np.float64(0.4489659303971375),
 'mrr@10': np.float64(0.4956672156988446),
 'ndcg@10': np.float64(0.3721611040249473)}

In [46]:
inspect_recommendations(
    user_id=full_grouped_data.loc[full_grouped_data['split'] == 'test', 'user_id'].iloc[11],
    df=full_grouped_data,
    id2recipe=id2recipe,
    id2item=id2item,
    preds='slim_preds',
    split='test'
)

RECIPE: https://www.povarenok.ru/recipes/show/8227/ (User ID: 83) | split: test
Observed Ingredients (5)
Чеснок, Морковь, Масло растительное, Помидор, Лук репчатый

Ground Truth Ingredients (2)
Баклажан, Перец болгарский

Top Recommended Ingredients (20)
Перец болгарский, Картофель, Зелень, Вода, Специи, Петрушка, Соевый соус, Перец чили, Сыр твердый, Рис, Баклажан, Лист лавровый, Яйцо куриное, Томатная паста, Капуста белокочанная, Кабачок, Уксус, Базилик, Майонез, Шампиньоны



In [47]:
random_test_user_id = full_grouped_data.loc[full_grouped_data['split'] == 'test', 'user_id'].sample(1).iloc[0]
inspect_recommendations(
    user_id=random_test_user_id,
    df=full_grouped_data,
    id2recipe=id2recipe,
    id2item=id2item,
    preds='ease_preds',
    split='test'
)

RECIPE: https://www.povarenok.ru/recipes/show/23288/ (User ID: 79274) | split: test
Observed Ingredients (4)
Масло растительное, Помидор, Капуста пекинская, Перец болгарский

Ground Truth Ingredients (3)
Уксус, Специи, Лук репчатый

Top Recommended Ingredients (20)
Лук репчатый, Чеснок, Огурец, Баклажан, Морковь, Зелень, Уксус, Соевый соус, Сок лимона, Перец чили, Лук зеленый, Петрушка, Сыр твердый, Кабачок, Специи, Кукуруза, Дрожжи, Филе куриное, Лук красный, Грудка куриная



In [48]:
import optuna

val_mask = full_grouped_data['split'] == 'val'
tqdm.pandas()

def objective(trial):
    l1_reg = trial.suggest_float('l1_reg', 1e-4, 1e-1)
    l2_reg = trial.suggest_float('l2_reg', 1e-4, 1e-1)
    w_candidate = train_slim(matrix_train, l1_reg=l1_reg, l2_reg=l2_reg)
    val_subset = full_grouped_data.loc[val_mask, ['user_id', 'support_items', 'holdout_items']].copy()
    val_subset['slim_val_preds'] = val_subset['support_items'].progress_apply(
        lambda interactions: get_slim_predictions(interactions, w_candidate)
    )
    return evaluate_model(val_subset, 'slim_val_preds', 'holdout_items', top_k=6)['hit_rate@6']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

[I 2025-11-15 11:36:51,841] A new study created in memory with name: no-name-ee2e358d-dbcd-46ec-9be1-800095a3f903


100%|██████████| 18496/18496 [00:09<00:00, 1881.41it/s]
[I 2025-11-15 11:39:49,469] Trial 0 finished with value: 0.4969723183391003 and parameters: {'l1_reg': 0.034785234635330266, 'l2_reg': 0.08705862199446206}. Best is trial 0 with value: 0.4969723183391003.
100%|██████████| 18496/18496 [00:02<00:00, 8192.70it/s]
[I 2025-11-15 11:42:55,454] Trial 1 finished with value: 0.2256704152249135 and parameters: {'l1_reg': 0.08045431074310089, 'l2_reg': 0.04178450986631001}. Best is trial 0 with value: 0.4969723183391003.
100%|██████████| 18496/18496 [00:02<00:00, 8745.93it/s]
[I 2025-11-15 11:45:37,707] Trial 2 finished with value: 0.4504757785467128 and parameters: {'l1_reg': 0.04228016225899976, 'l2_reg': 0.023272900156306116}. Best is trial 0 with value: 0.4969723183391003.
100%|██████████| 18496/18496 [00:03<00:00, 5061.54it/s]
[I 2025-11-15 11:47:38,094] Trial 3 finished with value: 0.3522923875432526 and parameters: {'l1_reg': 0.05873943470322269, 'l2_reg': 0.006079997746767759}. Best 

KeyboardInterrupt: 

In [108]:
study.best_trial.params['l1_reg'], study.best_trial.params['l2_reg']

(0.0003948790736704186, 0.007761665029654702)

In [49]:
w_slim = train_slim(interactions_matrix_train_val, l1_reg=0.02715746392003715,
l2_reg=0.054222437551529876)

100%|██████████| 973/973 [04:26<00:00,  3.65it/s]


In [50]:
tqdm.pandas()
for split_name in ['val', 'test']:
    split_mask = full_grouped_data['split'] == split_name
    if split_mask.any():
        full_grouped_data.loc[split_mask, 'slim_full_preds'] = (
            full_grouped_data.loc[split_mask, 'support_items']
            .progress_apply(lambda interactions: get_slim_predictions(interactions, w_slim))
        )

full_grouped_data.loc[full_grouped_data['split'] == 'test'].head()

100%|██████████| 24661/24661 [00:02<00:00, 9488.50it/s]


,user_id,split,support_items,holdout_items,ease_preds,slim_preds,slim_full_preds
98647,16,test,"[10, 8, 55, 63, 13]","[39, 62, 64]","[39, 4, 16, 3, 11, 18, 43, 2, 145, 70, 124, 48...","[39, 4, 18, 2, 16, 3, 11, 145, 48, 6, 70, 43, ...","[18, 39, 3, 4, 11, 16, 48, 46, 2, 0, 654, 653,..."
98648,21,test,"[15, 86, 85]","[24, 18]","[0, 24, 79, 65, 93, 4, 212, 142, 151, 159, 450...","[0, 4, 65, 35, 29, 24, 58, 14, 61, 66, 26, 31,...","[4, 0, 65, 642, 643, 644, 645, 646, 647, 648, ..."
98649,26,test,"[61, 15, 102, 4]","[89, 68]","[0, 65, 29, 142, 58, 151, 31, 24, 35, 22, 93, ...","[65, 0, 58, 142, 29, 31, 35, 24, 8, 66, 18, 2,...","[0, 18, 65, 58, 8, 2, 10, 24, 29, 142, 647, 64..."
98650,30,test,"[4, 18, 70, 39, 10]","[15, 107]","[13, 8, 0, 65, 2, 58, 24, 16, 15, 38, 3, 164, ...","[13, 3, 15, 24, 8, 0, 2, 58, 48, 16, 65, 46, 3...","[3, 15, 0, 65, 24, 13, 58, 11, 8, 16, 2, 46, 4..."
98651,31,test,"[48, 18, 16, 108, 109]","[54, 111, 110]","[68, 11, 55, 10, 3, 46, 111, 49, 54, 39, 77, 2...","[10, 3, 68, 13, 4, 2, 24, 11, 39, 46, 49, 77, ...","[10, 3, 4, 24, 39, 11, 49, 46, 77, 653, 652, 0..."


In [60]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='slim_full_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.5850127732046552,
 'precision@10': np.float64(0.08028871497506183),
 'recall@10': np.float64(0.2863007943266246),
 'mrr@10': np.float64(0.37740283643024813),
 'ndcg@10': np.float64(0.2536324234264217)}

### toppopular as baseline

In [56]:
class TopPopular:
    def __init__(self):
        self.recommendations = None
        self.trained = False

    def fit(self, df: pd.DataFrame, item_col: str = "item_id") -> None:
        self.recommendations = df[item_col].value_counts().index.to_numpy()
        self.trained = True

    def predict(self, user_ids: list, top_k: int = 20) -> list:
        if not self.trained:
            raise RuntimeError("You must fit the model before making predictions.")
        
        top_k_recs = self.recommendations[:top_k].tolist()
        return [top_k_recs] * len(user_ids)

toppop_model = TopPopular()
toppop_model.fit(train_df, item_col='item_id')

In [62]:
target_mask = full_grouped_data['split'] == 'test'
target_index = full_grouped_data.index[target_mask]

pop_predictions = toppop_model.predict(
    user_ids=full_grouped_data.loc[target_mask, 'user_id'],
    top_k=20
)

full_grouped_data.loc[target_index, 'toppop_preds'] = pd.Series(
    pop_predictions,
    index=target_index
)

In [63]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='toppop_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.6525282835245935,
 'precision@10': np.float64(0.08741332468269737),
 'recall@10': np.float64(0.31932189376838827),
 'mrr@10': np.float64(0.2967502025883681),
 'ndcg@10': np.float64(0.2302394463934872)}

In [115]:
full_grouped_data.head()

,user_id,train_interactions,val_interactions,test_interactions,ease_preds,ease_full_preds,slim_preds,slim_full_preds,toppop_preds
0,1,"[6, 3, 5, 7]",[4],"[2, 8]","[18, 49, 55, 11, 145, 8, 64, 77, 68, 4, 46, 12...","[18, 49, 55, 11, 8, 145, 77, 64, 2, 159, 68, 1...","[18, 55, 145, 11, 49, 4, 122, 8, 71, 77, 68, 6...","[18, 55, 11, 49, 145, 8, 4, 122, 68, 71, 64, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
1,2,"[11, 8, 10]",[3],[9],"[3, 39, 2, 4, 18, 46, 13, 16, 55, 48, 9, 70, 1...","[2, 4, 39, 3, 46, 13, 18, 55, 16, 9, 70, 48, 1...","[4, 18, 2, 39, 3, 46, 13, 55, 16, 70, 48, 43, ...","[18, 4, 2, 3, 39, 46, 13, 16, 55, 48, 70, 9, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
2,3,"[0, 2, 16, 18, 13, 10]","[12, 17]","[14, 15]","[4, 39, 3, 15, 8, 11, 48, 58, 151, 24, 181, 70...","[4, 39, 3, 15, 11, 48, 151, 8, 181, 43, 58, 70...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 43, 70,...","[4, 39, 3, 15, 8, 11, 48, 151, 24, 58, 70, 43,...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
3,4,"[19, 20, 22]",[23],[21],"[4, 24, 67, 66, 15, 159, 23, 65, 29, 0, 18, 73...","[24, 4, 67, 159, 66, 23, 15, 18, 29, 27, 61, 7...","[67, 24, 15, 159, 65, 4, 23, 73, 27, 29, 150, ...","[67, 4, 15, 65, 159, 23, 24, 73, 27, 150, 29, ...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."
4,5,"[24, 25, 4, 29, 15]","[28, 22]","[26, 27]","[18, 65, 0, 151, 10, 35, 58, 22, 66, 61, 31, 1...","[65, 18, 0, 151, 35, 10, 22, 58, 66, 61, 31, 1...","[18, 0, 65, 35, 151, 58, 22, 66, 61, 31, 142, ...","[18, 0, 65, 35, 58, 151, 22, 66, 31, 61, 10, 1...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 11, 13..."


In [116]:
top_20_ids = pop_predictions[0][:20]

print("TopPopular 20")
for item_id in top_20_ids:
    print(id2item.get(item_id))

TopPopular 20
Масло растительное
Яйцо куриное
Лук репчатый
Масло сливочное
Чеснок
Вода
Морковь
Молоко
Сметана
Майонез
Сыр твердый
Помидор
Картофель
Разрыхлитель теста
Зелень
Сливки
Перец болгарский
Соевый соус
Специи
Сок лимона


In [ ]:
inspect_recommendations(
    user_id=full_grouped_data.loc[full_grouped_data['split'] == 'test', 'user_id'].iloc[0],
    df=full_grouped_data,
    id2recipe=id2recipe,
    id2item=id2item,
    preds='toppop_preds',
    split='test'
)

RECIPE: https://www.povarenok.ru/recipes/show/142765/ (User ID: 145)

--- Ingredients in Training Set (4) ---
Молоко, Какао, Ягода, Сливки

--- Ground Truth Ingredients in Test Set (2) ---
Банан, Крупа манная

--- Top Recommended Ingredients (20) ---
Масло растительное, Яйцо куриное, Лук репчатый, Масло сливочное, Чеснок, Вода, Морковь, Молоко, Сметана, Майонез, Сыр твердый, Помидор, Картофель, Разрыхлитель теста, Зелень, Сливки, Перец болгарский, Соевый соус, Специи, Сок лимона



### Tuning itemKNN

In [ ]:
import numpy as np
import scipy.sparse as sps

def bm25_weight(X: sps.spmatrix, k1: float = 1.2, b: float = 0.75) -> sps.csr_matrix:
    X = X.tocoo().astype(np.float32)
    num_users = X.shape[0]

    df = np.bincount(X.col, minlength=X.shape[1])
    idf = np.log((num_users - df + 0.5) / (df + 0.5))
    idf = np.maximum(idf, 0.0)

    row_counts = np.bincount(X.row, minlength=X.shape[0]).astype(np.float32)
    avg_length = row_counts.mean() + 1e-9
    length_norm = k1 * ((1 - b) + b * row_counts[X.row] / avg_length)

    data = X.data * (k1 + 1.0) / (X.data + length_norm + 1e-9)
    data *= idf[X.col]

    return sps.csr_matrix((data, (X.row, X.col)), shape=X.shape)


def train_itemknn(user_item_matrix: sps.spmatrix, topk: int = 200, shrink: float = 100.0, use_binary: bool = True,
                  bm25_k1: float = 1.2, bm25_b: float = 0.75) -> np.ndarray:

    X = user_item_matrix.tocsr().astype(np.float32)
    X = bm25_weight(X, k1=bm25_k1, b=bm25_b)

    C = (X.T @ X).astype(np.float32).toarray()  # items x items

    norms = np.sqrt(np.asarray(X.power(2).sum(axis=0)).ravel() + 1e-9)  # (items,)
    denom = (norms[:, None] * norms[None, :]) + 1e-8
    cos_sim = C / denom

    if use_binary:
        Xb = X.copy()
        Xb.data[:] = 1.0
        co_counts = (Xb.T @ Xb).astype(np.float32).toarray()
    else:
        co_counts = C
    alpha = co_counts / (co_counts + shrink + 1e-9)
    cos_sim *= alpha

    np.fill_diagonal(cos_sim, 0.0)

    if topk is not None and topk > 0 and topk < cos_sim.shape[1]:
        k = int(topk)
        idx = np.argpartition(-cos_sim, kth=k, axis=1)
        rows = np.arange(cos_sim.shape[0])[:, None]
        mask = np.ones_like(cos_sim, dtype=bool)
        topk_idx = idx[:, :k]
        mask[rows, topk_idx] = False
        cos_sim[mask] = 0.0

    return cos_sim.astype(np.float32)



preds_itemknn = train_itemknn(matrix_train, topk=200, shrink=100.0, use_binary=True)

tqdm.pandas()
test_mask = full_grouped_data['split'] == 'test'
full_grouped_data.loc[test_mask, 'itemknn_preds'] = (
    full_grouped_data.loc[test_mask, 'support_items']
    .progress_apply(lambda interactions: get_preds(interactions, item2id, id2item, preds_itemknn))
)
full_grouped_data.loc[test_mask].tail()

100%|██████████| 24661/24661 [00:08<00:00, 2984.90it/s]


,user_id,split,support_items,holdout_items,ease_preds,slim_preds,slim_full_preds,toppop_preds,itemknn_preds
123303,146522,test,"[55, 54, 3, 46, 68]","[210, 49, 18]","[18, 11, 8, 16, 49, 145, 7, 71, 122, 109, 77, ...","[18, 11, 8, 77, 16, 71, 49, 10, 145, 39, 7, 9,...","[18, 10, 11, 0, 641, 642, 643, 644, 645, 646, ...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 13, 11...","[18, 11, 10, 8, 39, 49, 16, 71, 77, 122, 145, ..."
123304,146523,test,"[16, 58, 290, 18, 15, 48, 11, 4, 3]","[213, 379, 6, 142]","[10, 0, 2, 65, 46, 8, 13, 24, 142, 213, 9, 151...","[10, 0, 2, 46, 65, 24, 13, 142, 122, 71, 49, 9...","[10, 0, 65, 24, 8, 39, 2, 49, 46, 29, 77, 142,...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 13, 11...","[10, 39, 2, 0, 13, 24, 46, 65, 8, 49, 142, 77,..."
123305,146527,test,"[288, 0, 4, 66, 24]","[15, 341]","[15, 18, 65, 151, 142, 29, 31, 202, 14, 169, 5...","[15, 65, 18, 151, 142, 29, 58, 31, 2, 61, 8, 1...","[15, 18, 65, 58, 8, 2, 10, 29, 142, 647, 646, ...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 13, 11...","[15, 65, 18, 151, 29, 31, 58, 142, 10, 61, 35,..."
123306,146535,test,"[18, 329, 0, 68]","[159, 49]","[4, 10, 151, 15, 49, 3, 122, 48, 232, 145, 16,...","[4, 15, 151, 10, 3, 24, 49, 64, 67, 159, 232, ...","[4, 10, 3, 24, 15, 39, 11, 49, 46, 16, 77, 652...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 13, 11...","[4, 15, 10, 151, 3, 24, 65, 39, 49, 11, 16, 2,..."
123307,146547,test,"[278, 122, 36, 4]","[210, 49, 13]","[8, 15, 18, 6, 0, 10, 13, 58, 11, 55, 145, 38,...","[15, 18, 0, 3, 65, 8, 145, 58, 142, 2, 6, 11, ...","[15, 18, 0, 65, 58, 8, 2, 10, 24, 29, 142, 653...","[18, 4, 10, 15, 3, 24, 39, 0, 58, 8, 2, 13, 11...","[15, 18, 10, 0, 8, 65, 3, 58, 2, 13, 145, 39, ..."


In [65]:
evaluate_model(
    df=full_grouped_data[full_grouped_data['split'] == 'test'],
    preds_col='itemknn_preds',
    gt_col='holdout_items',
    top_k=10
)

{'hit_rate@10': 0.7972101699038968,
 'precision@10': np.float64(0.1269048294878553),
 'recall@10': np.float64(0.4596644145405347),
 'mrr@10': np.float64(0.4900960677581658),
 'ndcg@10': np.float64(0.374254179081696)}